# <center>Workflow for on-line GC and HPLC analysis in flow chemistry</center>


---
## Section 0: Imports, Paths, and Logging
---

In this section all the necessary python packages are imported, the path to this notebook and the logger for this notebook is set up.

In [1]:
# Activate autoreload to keep on track with changing modules #
%reload_ext autoreload
%autoreload 2

# Import standard libraries #
import numpy as np
import logging
import json
from pathlib import Path

# Import tools for data processing and analysis #
from datamodel.tools import initialize_dataset
from datamodel.tools import reading_raw_data_widget
from datamodel.tools import analyzing_raw_data_widget


In [2]:
# Define paths for loggin output #
root                = Path.cwd()
logging_config_path = root / "datamodel_b07_tc/tools/logging/config.json"

# Read in logger specs and configurate logger (set name to current notebook) #
with open(logging_config_path) as logging_config_json: logging.config.dictConfig( json.load( logging_config_json ) )
logger = logging.getLogger(__name__)

# Set the level of thid-party logger to avoid dumping too much information #
for logger_ in ['markdown_it', 'h5py', 'numexpr', 'git']: logging.getLogger(logger_).setLevel('WARNING')

In [ ]:
### Fragen an Jan:

# Wenn ich dataset wieder einlese, ist das format nicht mehr das was es war davor --> kann ich es wiederherstellen (float, datetime etc)

---
## Section 1: Dataset and raw data
---
In this section the dataset as well as the to analyze raw data is choosen

In [3]:
git_path = 'https://github.com/FAIRChemistry/datamodel_b07_tc.git'
branch   = 'samir_develop'

id = initialize_dataset()
id.write_dataset(root, git_path, branch)

Saving dataset!


In [4]:
## Definition of basic meta data of the project ##

id.title.value        = 'Electrocatalytic CO2-reduction on carbon'
id.description.value  = 'The aim of this project is to blablabla'

# List with authors and their affiliation #
id.authors.value      = 'Richard Schömig, Maximilian Schmidt' 
id.affiliations.value = 'University of Stuttgart, University of Stuttgart'
id.dataset_text.value = "test"

In [5]:
## Search for dataset and raw data ##

rrdw = reading_raw_data_widget()
rrdw.choose_data(root)

In [7]:
#str(root) = "c:/Users/darouich/OneDrive/Dokumente/"

e_chem = str(root)+'/data/Rohdaten/01_EChem/CAD14-Cu@AB/GSTATIC.DTA'
mfm    = str(root)+'/data/Rohdaten/03_MFM/CAD14-Cu@AB/Bench-2h-GSS_CAD14-Cu@AB_200_50c_24h_truncated.csv'
gc     = [str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/REPORT01.CSV']
calib  = str(root)+'/data/calibration/calibration.json'
correc = str(root)+'/data/correction_factors/correction_factors.json'
farada = str(root)+'/data/faraday_coefficients/faraday_coefficients.json'

rrdw.Echem_files.value = [e_chem]
rrdw.MFM_files.value   = [mfm]
rrdw.GC_files.value    = gc
rrdw.calib_files.value = [calib]
rrdw.correction_files.value = [correc]
rrdw.faraday_files.value    = [farada]

---
## Section 2: Analysis of data
---
In this section the raw data of the above choosen dataset is analyzed

In [6]:
# Provide a typical retention time dictionary to pre assign retention times 

typical_retention_time = {"Hydrogen": 1.7, "Carbon dioxide": 3.0, "Carbon monoxide": 13.6, 
                          "Methane": 3.6, "Ethene": 6.0, "Ethane": 7.1}

ardw = analyzing_raw_data_widget()
ardw.choose_experiment( datamodel = rrdw.datamodel, 
                        dataset_path = rrdw.dataset_dropdown.value, 
                        typical_retention_time = typical_retention_time)


Starting the postprocessing



TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [43]:
from datetime import datetime
from datamodel.core import Dataset, Experiment
import pandas as pd
dataset, lib = rrdw.datamodel
c = Experiment( **rrdw.dataset.experiments[0].__dict__ )

a,b = c.volumetric_flow_time_course
#--> implement that when rereading the dataset the correct data types e.g float, datetime, etc are restorted !!!!

d = [ pd.to_datetime(timestamp, format="%Y-%m-%d %H:%M:%S").to_pydatetime()
                   for timestamp in a ]
if type(d[0]) == datetime:
    print("yers")

type( Dataset(**dataset.__dict__).experiments[0].measurements[2].metadata[0].value )

yers


str

---
## Section 3: Upload of data to DaRUS
---
In this section the processed, as well as the raw data is uploaded to DaRUS

In [75]:
from pyDaRUS import Citation
from pyDaRUS import Dataset
from pyDaRUS.metadatablocks.citation import SubjectEnum, IdentifierScheme

subject_list = [ subject.value for subject in SubjectEnum]

# Initialize the metadatablock
citation = Citation()

citation.title = "My Title"

citation.add_description(text="test")
citation.add_author(name="Samir", affiliation="ITT")
citation.add_contact(name="Samir", email="samir.darouich@itt.uni-stuttgart.de")

citation.subject = ["Engineering"]

citation.add_grant_information( grant_agency="DFG", grant_number="358283783 - SFB 1333")
citation.add_project( name="test project", level=1 )
citation.depositor = "XXX"
citation.deposit_date = "XXX"


dataset = Dataset()
dataset.add_metadatablock(citation)

dataset.add_file(dv_path=".", local_path="my.file")

dataset.upload(dataverse_name="sfb1333-hansen-gross",
               DATAVERSE_URL="https://darus.uni-stuttgart.de",
               API_TOKEN="4afecd82-c92d-4935-b786-2225af43531e")

Attribute project not valid for import (dv_up).


Exception: Validation Failed: Point of Contact Name is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Description Text is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Project Level is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Funding Information Identifier is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Project Name is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Deposit Date is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Author Name is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Depositor is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Point of Contact E-mail is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Funding Information Agency is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]